In [1]:
# import those libraries in a virtual environment
from PIL import Image
from transformers import AutoProcessor
from transformers import AutoModelForUniversalSegmentation
import torch
from collections import defaultdict
import matplotlib.pyplot as plt
import pickle 
import matplotlib.patches as mpatches
import time
import tqdm
import datetime
import glob
import os
import cv2
import argparse
import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = 'cityscapes'

In [3]:
def draw_panoptic_segmentation(parent_dir, filename, model, segmentation, segments_info):
    # get the used color map
    # viridis = cm.get_cmap('viridis', torch.max(segmentation))
    if len(segments_info) == 0:
        return

    viridis = plt.get_cmap('viridis', torch.max(segmentation))
    fig = plt.figure()
    fig.set_size_inches(480/200, 270/200)
    ax = fig.add_axes([0, 0, 1, 1])  # span the whole figure
    ax.imshow(segmentation.cpu())
    instances_counter = defaultdict(int)
    handles = []
    # for each segment, draw its legend
    for segment in segments_info:
        segment_id = segment['id']
        segment_label_id = segment['label_id']
        segment_label = model.config.id2label[segment_label_id]
        label = f"{segment_label}-{instances_counter[segment_label_id]}"
        instances_counter[segment_label_id] += 1
        color = viridis(segment_id)
        handles.append(mpatches.Patch(color=color, label=label))
    # ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5))

    ax.set_axis_off()
    plt.savefig(os.path.join(parent_dir, 'frames_pred_%s/%s' % (model_name, os.path.basename(filename))), dpi=200)
    plt.close(fig)

In [7]:
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
# open file 
parent_dir = '../../0_data/0_raw_extracted_frames'

batch_size = 6

if not os.path.exists(os.path.join(parent_dir, 'frames_pred_%s' % model_name)):
    os.makedirs(os.path.join(parent_dir, 'frames_pred_%s' % model_name))

if model_name == 'cityscapes':
    model = AutoModelForUniversalSegmentation.from_pretrained("shi-labs/oneformer_cityscapes_swin_large")
    processor = AutoProcessor.from_pretrained("shi-labs/oneformer_cityscapes_swin_large")
elif model_name == 'ade20k':
    model = AutoModelForUniversalSegmentation.from_pretrained("shi-labs/oneformer_ade20k_swin_large")
    processor = AutoProcessor.from_pretrained("shi-labs/oneformer_ade20k_swin_large")
elif model_name == 'coco':
    model = AutoModelForUniversalSegmentation.from_pretrained("shi-labs/oneformer_coco_swin_large")
    processor = AutoProcessor.from_pretrained("shi-labs/oneformer_coco_swin_large")
else:
    raise ValueError('model not found')

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

all_images = glob.glob(os.path.join(parent_dir, '*.png'))
all_images.sort()

segmentation_folder = 'segmentation_pred_%s' % model_name
if not os.path.exists(os.path.join(parent_dir, segmentation_folder)):
    os.makedirs(os.path.join(parent_dir, segmentation_folder))

# read every batch images into disk
for i in tqdm.tqdm(range(0, len(all_images), batch_size)):
    img_batch = []
    filename_batch = []
    for j in range(i, min(i+batch_size, len(all_images))):
        img = Image.open(all_images[j])
        # resize the image to (640, 360)
        img = img.resize((640, 360), Image.ANTIALIAS)
        filename_batch.append(all_images[j])
        img_batch.append(img)
    
    # convert the list of images to a torch batch
    # img_batch_tensor = torch.cat([ToTensor()(img).unsqueeze_(0) for img in img_batch], dim = 0)

    panoptic_inputs = processor(images=img_batch, task_inputs=["panoptic"] * len(img_batch), return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = model(**panoptic_inputs.to(device))

    # post-process the predictions
    panoptic_segmentation = processor.post_process_panoptic_segmentation(outputs, target_sizes=[img.size[::-1]] * len(img_batch))
    for item, filename in zip(panoptic_segmentation, filename_batch):
        # draw the output 
        # TODO comment the following line if not drawing
        # draw_panoptic_segmentation(parent_dir, filename, model, **item)

        # save the output predictions
        # comment the following line if not saving the predictions
        with open(os.path.join(parent_dir, segmentation_folder, os.path.basename(filename).split('.')[0] + '.pkl'), 'wb') as f:
            pickle.dump(item, f)

    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))


2024-08-19 12:44:57


  0%|          | 1/1583 [02:00<53:02:44, 120.71s/it]

2024-08-19 12:47:02


`label_ids_to_fuse` unset. No instance will be fused.
  0%|          | 2/1583 [04:01<53:01:27, 120.74s/it]

2024-08-19 12:49:03


...

2024-08-21 14:29:36


`label_ids_to_fuse` unset. No instance will be fused.
100%|█████████▉| 1582/1583 [49:46:21<01:47, 107.09s/it]

2024-08-21 14:31:23


`label_ids_to_fuse` unset. No instance will be fused.
100%|██████████| 1583/1583 [49:46:47<00:00, 113.21s/it]

2024-08-21 14:31:49
